In [40]:


from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Reshape, LeakyReLU, Activation, Input, Softmax
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import resnet50

In [41]:
class Yolo(Model):
    def get_config(self):
        super(Yolo, self).get_config()

    def __init__(self):
        super(Yolo, self).__init__()
        self._con1 = Conv2D(filters=64, kernel_size=(7, 7), strides=2, padding="same")
        self._a1 = LeakyReLU()
        self._mp1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding="same")

        self._con2 = Conv2D(filters=128, kernel_size=(1, 1), strides=1, padding="same")
        self._con3 = Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding="same")
        self._a2 = LeakyReLU()
        self._mp2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding="same")

        self._con4 = Conv2D(filters=512, kernel_size=(1, 1), strides=1, padding="same")
        self._con5 = Conv2D(filters=1024, kernel_size=(3, 3), strides=2, padding="same")
        self._a3 = LeakyReLU()
        self._mp3 = MaxPooling2D(pool_size=(2, 2), strides=2, padding="same")

        self._fl = Flatten()
        self._fc1 = Dense(512)
        self._fca1 = LeakyReLU()
        self._dr = Dropout(0.2)
        self._fc2 = Dense(7 * 7 * 15)
        self._reshape = Reshape((7, 7, 15))


    def call(self, inputs, training=None, mask=None):
        output = self._con1(inputs)
        output = self._a1(output)
        output = self._mp1(output)

        output = self._con2(output)
        output = self._con3(output)
        output = self._a2(output)
        output = self._mp2(output)

        output = self._con4(output)
        output = self._con5(output)
        output = self._a3(output)
        output = self._mp3(output)

        output = self._fl(output)
        output = self._fc1(output)
        output = self._fca1(output)
        output = self._dr(output)
        output = self._fc2(output)
        output = self._reshape(output)
        return output

    @staticmethod
    def yolo_loss(y_real, y_fake):
        bounding_box = Activation("sigmoid")(y_fake[:, :, :, :5])


In [42]:
y = Yolo()
y.build(input_shape=(None, 224, 224, 3))
y.call(Input((224, 224, 3)))
y.summary()

Model: "yolo_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 112, 112, 64)      9472      
_________________________________________________________________
leaky_re_lu_52 (LeakyReLU)   (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 56, 56, 128)       8320      
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 56, 56, 512)       590336    
_________________________________________________________________
leaky_re_lu_53 (LeakyReLU)   (None, 56, 56, 512)       0         
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 28, 28, 512)       0   